# LinkedIn Job Postings

## ADS 508 Impacting the Business with a Distributed Data Science Pipeline

In [2]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import sagemaker
from pyathena import connect
import awswrangler as wr

import warnings
warnings.filterwarnings('ignore')

## Data Ingestion

### CSV to S3

In [3]:
# Setup boto3 session parameters
session = sagemaker.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

# Establish connection
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [4]:
# Set S3 Source Location (Public bucket)
s3_public_path = "s3://linkedin-postings"

In [5]:
%store s3_public_path

Stored 's3_public_path' (str)


In [6]:
# Set S3 Destination Location (Private bucket)
s3_private_path = "s3://{}/linkedin_data".format(bucket)
print(s3_private_path)

s3://sagemaker-us-east-1-937572952481/linkedin_data


In [7]:
%store s3_private_path

Stored 's3_private_path' (str)


In [8]:
# Copy data from Public S3 bucket to Private S3 bucket
!aws s3 cp --recursive $s3_public_path/ $s3_private_path/ --exclude "*" --include "postings/postings.csv"
!aws s3 cp --recursive $s3_public_path/ $s3_private_path/ --exclude "*" --include "salaries/salaries.csv"
!aws s3 cp --recursive $s3_public_path/ $s3_private_path/ --exclude "*" --include "job_skills/job_skills.csv"

copy: s3://linkedin-postings/postings/postings.csv to s3://sagemaker-us-east-1-937572952481/linkedin_data/postings/postings.csv
copy: s3://linkedin-postings/salaries/salaries.csv to s3://sagemaker-us-east-1-937572952481/linkedin_data/salaries/salaries.csv
copy: s3://linkedin-postings/job_skills/job_skills.csv to s3://sagemaker-us-east-1-937572952481/linkedin_data/job_skills/job_skills.csv


In [9]:
# Check files are copied successfully to private bucket
!aws s3 ls $s3_private_path/

                           PRE job_skills/
                           PRE postings/
                           PRE salaries/


### Create Athena Database 

In [10]:
ingest_create_athena_db_passed = False

In [11]:
ingest_create_athena_table_passed = False

In [12]:
database_name = "linkedin_data"

In [13]:
# Set S3 staging directory - a temporary directory for Athena queries
s3_staging_dir = "s3://{}/athena/staging".format(bucket)

In [14]:
# Connect to staging directory
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [15]:
# Create Database
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)

pd.read_sql(statement, conn)

""


### Verify database has been created

In [16]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,default
1,dsoaws
2,linkedin_data


In [17]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

### Create Athena Tables from CSV Files

In [18]:
table_name = 'postings'
postings_path = "s3://{}/linkedin_data/postings/".format(bucket)

# SQL statement to execute the postings table
statement = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
        job_id int,
        company_name string,
        title string,
        description string,
        max_salary float,
        pay_period string,
        location string,
        company_id float,
        views float,
        med_salary float,
        min_salary float,
        formatted_work_type string,
        applies float,
        original_listed_time float,
        remote_allowed float,
        job_posting_url string,
        application_url string,
        application_type string,
        expiry float,
        closed_time float,
        formatted_experience_level string,
        skills_desc string,
        listed_time float,
        posting_domain string,
        sponsored int,
        work_type string,
        currency string,
        compensation_type string,
        normalized_salary float,
        zip_code int,
        fips int
    ) 
    ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
    LOCATION '{}' 
    TBLPROPERTIES ('skip.header.line.count'='1')
    """.format(database_name, table_name, postings_path)

# Execute statement
pd.read_sql(statement, conn)
print("Created postings table")

Created postings table


In [19]:
table_name_2 = "salaries"
salaries_path = "s3://{}/linkedin_data/salaries/".format(bucket)

# SQL statement to execute the postings table
statement = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
        salary_id int,
        job_id int,
        max_salary float,
        med_salary float,
        min_salary float,
        pay_period string,
        currency string,
        compensation_type string
    ) 
    ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
    LOCATION '{}' 
    TBLPROPERTIES ('skip.header.line.count'='1')
    """.format(database_name, table_name_2, salaries_path)

# Execute statement
pd.read_sql(statement, conn)
print("Created salaries table")

Created salaries table


In [20]:
table_name_3 = "job_skills"
job_skills_path = "s3://{}/linkedin_data/job_skills/".format(bucket)

# SQL statement to execute the postings table
statement = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
        job_id int,
        skill_abr string
    ) 
    ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
    LOCATION '{}' 
    TBLPROPERTIES ('skip.header.line.count'='1')
    """.format(database_name, table_name_3, job_skills_path)

# Execute statement
pd.read_sql(statement, conn)
print("Created job_skills table")

Created job_skills table


### Verify tables have been created successfully

In [21]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,tab_name
0,job_skills
1,postings
2,salaries


In [22]:
if table_name in df_show.values:
    ingest_create_athena_table_passed = True

In [25]:
# Show the first few rows to ensure that the data looks correct
statement = """SELECT * FROM {}.{} LIMIT 5""".format(database_name, table_name)

print(statement)

SELECT * FROM linkedin_data.postings LIMIT 5


In [26]:
df = pd.read_sql(statement, conn)
df

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips


In [72]:
# View postings table to check the data looks correct
# postings_df = pd.read_sql(statement, conn)
postings_df = wr.athena.read_sql_query(sql="SELECT * FROM {} LIMIT 5".format("postings"), database=database_name)
postings_df

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips


In [47]:
# View salaries table to check the data looks correct
# salaries_df = pd.read_sql(statement, conn)
salaries_df = wr.athena.read_sql_query(sql="SELECT * FROM {} LIMIT 5".format("salaries"), database=database_name)
salaries_df

,salary_id,job_id,max_salary,med_salary,min_salary,pay_period,currency,compensation_type


In [48]:
# View job skills table to check the data looks correct
# job_skills_df = pd.read_sql(statement, conn)
job_skills_df = wr.athena.read_sql_query(sql="SELECT * FROM {} LIMIT 5".format("job_skills"), database=database_name)
job_skills_df

,job_id,skill_abr


## Data Exploration

### postings EDA

In [ ]:
# Basic stats
print(postings_df.describe())
print(postings_df.info())

In [ ]:
# Data types
postings_data_types = postings_df.dtypes
print(postings_data_types)

In [ ]:
# Missing values
postings_df.isnull().sum()

In [ ]:
# Duplicates
postings_df.duplicated().sum()

In [ ]:
# Summary statistics
print(f"Unique Job Titles: {postings_df['title'].nunique()}")
print(f"Unique Companies: {postings_df['company_name'].nunique()}")

In [ ]:
# Visualize top 10 most common job titles
plt.figure(figsize=(12,6))
postings_df['title'].value_counts().head(10).plot(kind='bar', color='green')
plt.xlabel("Job Title")
plt.ylabel("Number of Postings")
plt.title("Top 10 Most Common Job Titles")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Posting trends
postings_df['listed_time'] = postings_df.to_datetime(postings_df['listed_time'], unit='s')  # Convert Unix timestamp to datetime
postings_df.set_index('listed_time', inplace=True)

# Resample by month to see posting trends
postings_df.resample('M').size().plot(figsize=(12, 5), title="Job Posting Trends Over Time")
plt.ylabel("Number of Postings")
plt.show()

### salaries EDA

In [ ]:
# Basic stats
print(salaries_df.describe())
print(salaries_df.info())

In [ ]:
# Data types
salaries_data_types = salaries_df.dtypes
print(salaries_data_types)

In [ ]:
# Missing values
salaries_df.isnull().sum()

In [ ]:
# Duplicates
salaries_df.duplicated().sum()

In [ ]:
# Salary distributions
print("Max Salary: ", salaries_df['max_salary'].describe())
print("Min Salary: ", salaries_df['min_salary'].describe())
print("Median Salary: ", salaries_df['med_salary'].describe())
# print(df[['min_salary', 'med_salary', 'max_salary', 'normalized_salary']].describe())

In [ ]:
# Visualize salary distribution
plt.figure(figsize=(10,5))
sns.histplot(df['normalized_salary'], bins=30, kde=True, color="blue")
plt.xlabel("Salary")
plt.ylabel("Frequency")
plt.title("Salary Distribution")
plt.show()

### job_skills EDA

In [ ]:
# Basic stats
print(job_skills_df.describe())
print(job_skills_df.info())

In [ ]:
# Data types
job_skills_data_types = job_skills_df.dtypes
print(job_skills_data_types)

In [ ]:
# Missing values
job_skills_df.isnull().sum()

In [ ]:
# Duplicates
job_skills_df.duplicated().sum()

In [ ]:
# Summary statistics
print(f"Unique Job Skills: {job_skills_df['skill_abr'].nunique()}")

In [ ]:
from collections import Counter
from wordcloud import WordCloud

# Concatenate all skills descriptions
all_skills = " ".join(str(skill) for skill in job_skills_df['skills_abr'].dropna())

# Count word frequencies
word_counts = Counter(all_skills.split())

# Generate WordCloud
wordcloud = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(word_counts)

# Visualize most common job skills in job descriptions
plt.figure(figsize=(12,6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Most Common Skills in Job Postings")
plt.show()

### Bias Analysis

In [ ]:
# Detecting Skewed Data in Job Roles shown above
# Detecting salary bias shown above

In [ ]:
# Detecting Biased Language in Job Descriptions (might favor aggressive hiring cultures)

# List of biased words to check
biased_words = ["dominant", "aggressive", "competitive", "strong leader"]

# Count biased words in descriptions
word_counts = Counter(" ".join(str(desc) for desc in postings_df['description'].dropna()).lower().split())

# Filter for biased words
biased_word_freq = {word: word_counts[word] for word in biased_words if word in word_counts}

# Visualize word frequency
plt.figure(figsize=(10,5))
sns.barplot(x=list(biased_word_freq.keys()), y=list(biased_word_freq.values()), palette="Reds_r")
plt.xlabel("Biased Words")
plt.ylabel("Frequency")
plt.title("Occurrence of Potentially Biased Words in Job Descriptions")
plt.show()